In [ ]:
import cv2
import mediapipe as mp
import serial
import time
import math
ser = serial.Serial('COM9', 115200, timeout=1)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)
def calculate_angle(p1, p2, p3):
    """ Calculate angle between three points. """
    v1 = (p1.x - p2.x, p1.y - p2.y)
    v2 = (p3.x - p2.x, p3.y - p2.y)
    dot_product = v1[0] * v2[0] + v1[1] * v2[1]
    magnitude_v1 = math.sqrt(v1[0]**2 + v1[1]**2)
    magnitude_v2 = math.sqrt(v2[0]**2 + v2[1]**2)
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    angle = math.degrees(math.acos(min(1.0, max(-1.0, cos_angle))))
    return angle

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb_frame)
    
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            finger_tips_ids = [8, 12, 16, 20]
            count_fingers = sum([hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[tip_id - 3].y for tip_id in finger_tips_ids])  
            thumb_angle = calculate_angle(
                hand_landmarks.landmark[2], 
                hand_landmarks.landmark[3], 
                hand_landmarks.landmark[4]  
            )
            if thumb_angle > 160:  
                count_fingers += 1
            
            # Display the number of fingers on the frame
            cv2.putText(frame, str(count_fingers), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
            print("finger:", count_fingers)

            # Send finger count to Arduino via serial communication
            # ser.write(bytes(str(count_fingers), 'utf-8')) 
      #   ser.write(bytes('0', 'utf-8')) 

    # Show the frame
    cv2.imshow('Finger Count', frame)
    
    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
